In [2]:
#import libraries
import os
import random as rd
import tensorflow as tf
import matplotlib.pyplot as plt
import pyBigWig as pbg
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd
import matplotlib as mpl
import keras.backend as K
import pandas as pd
import logomaker as lm
import seaborn as sns

In [ ]:
#load all chrs
X_2L=np.load('/home/florian/projet/r6.16/seq.npz')['2L']
X_2R=np.load('/home/florian/projet/r6.16/seq.npz')['2R']
X_3L=np.load('/home/florian/projet/r6.16/seq.npz')['3L']
X_3R=np.load('/home/florian/projet/r6.16/seq.npz')['3R']
X_4=np.load('/home/florian/projet/r6.16/seq.npz')['4']
X_X=np.load('/home/florian/projet/r6.16/seq.npz')['X']
X_Y=np.load('/home/florian/projet/r6.16/seq.npz')['Y']

In [ ]:
# create scATAC values for each chr
cut=500
Y_2L=np.load('/home/florian/projet/scATACseq_14chr.npz')['2L'][0]
Y_2L[Y_2L >= cut] = cut
Y_2L=Y_2L/cut

Y_2R=np.load('/home/florian/projet/scATACseq_14chr.npz')['2R'][0]
Y_2R[Y_2R >= cut] = cut
Y_2R=Y_2R/cut

Y_3L=np.load('/home/florian/projet/scATACseq_14chr.npz')['3L'][0]
Y_3L[Y_3L >= cut] = cut
Y_3L=Y_3L/cut

Y_3R=np.load('/home/florian/projet/scATACseq_14chr.npz')['3R'][0]
Y_3R[Y_3R >= cut] = cut
Y_3R=Y_3R/cut

Y_4=np.load('/home/florian/projet/scATACseq_14chr.npz')['4'][0]
Y_4[Y_4 >= cut] = cut
Y_4=Y_4/cut

Y_X=np.load('/home/florian/projet/scATACseq_14chr.npz')['X'][0]
Y_X[Y_X >= cut] = cut
Y_X=Y_X/cut

Y_Y=np.load('/home/florian/projet/scATACseq_14chr.npz')['Y'][0]
Y_Y[Y_Y >= 80] = 80
Y_Y=Y_Y/80

In [ ]:
#load preds
pred2L=np.load('/home/florian/projet/models/preds_new_cut_weightless_501bp.npz')['pred2L']
pred2R=np.load('/home/florian/projet/models/preds_new_cut_weightless_501bp.npz')['pred2R']
pred3L=np.load('/home/florian/projet/models/preds_new_cut_weightless_501bp.npz')['pred3L']
pred3R=np.load('/home/florian/projet/models/preds_new_cut_weightless_501bp.npz')['pred3R']
pred4=np.load('/home/florian/projet/models/preds_new_cut_weightless_501bp.npz')['pred4']
predX=np.load('/home/florian/projet/models/preds_new_cut_weightless_501bp.npz')['predX']
predY=np.load('/home/florian/projet/models/preds_new_cut_weightless_501bp.npz')['predY']

In [ ]:
#load model
from keras.models import load_model
model_name='new_cut_weightless'
model2 = load_model('/home/florian/projet/models/'+ model_name +'/'+ model_name+ '.h5', compile=False)

In [ ]:
def compute_saliency_map(input_seq, model):
    # Compute the gradients of the output with respect to the input
    input_seq=tf.cast(input_seq,tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(input_seq)
        output = model(tf.expand_dims(input_seq,0))
    grads = tape.gradient(output, input_seq)
    

    # Compute the saliency map
    saliency_map = grads #tf.multiply(input_seq, grads)

    # Sum the saliency map over the channel dimension
    saliency_map = tf.reduce_sum(saliency_map, axis=-1)

    # # Normalize the saliency map
    # saliency_map = tf.math.abs(saliency_map)
    # saliency_map /= tf.reduce_max(saliency_map)

    # Return the saliency map
    return saliency_map

In [ ]:
def compute_saliency_channels(input_seq, model):
    # Compute the gradients of the output with respect to the input
    input_seq=tf.cast(input_seq,tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(input_seq)
        output = model(tf.expand_dims(input_seq,0))
    grads = tape.gradient(output, input_seq)

    # Compute the saliency map
    saliency_map = grads #tf.multiply(input_seq, grads)

    # # Normalize the saliency map
    # saliency_map = tf.math.abs(saliency_map)
    # saliency_map /= tf.reduce_max(saliency_map)

    # Return the saliency map
    return saliency_map

In [ ]:
def fast_pred(input,model):
    model=model
    return model(tf.expand_dims(tf.cast(input,tf.float32),0))

In [ ]:
def fast_pred_seq(input,window_start,model):
    model=model
    X=[]
    for i in range(window_start,window_start+2001):
        X.append(np.array(fast_pred(input[i-1000:i+1001],model))[0])
    return(X)

In [ ]:
def get_max(array,n):
    return ((-array).argsort()[:n])

def get_min(array,n):
    return (array.argsort()[:n])

In [ ]:
def mutation(window_start,chr, vals):
    mut=np.copy(chr)
    for i in vals:
        if 0 < i <= 2001:
            mut[i+window_start]=np.roll(mut[i+window_start],1)
        elif 2001 < i <=4002:
            mut[i+window_start-2001]=np.roll(mut[i+window_start-2001],1)
        elif 4002 < i <=6003:
            mut[i+window_start-4002]=np.roll(mut[i+window_start-4002],1)
        else:
            mut[i+window_start-6003]=np.roll(mut[i+window_start-6003],1)
    return mut

In [ ]:
def window_map(input,window_start,model):
    model=model
    Y=np.zeros(4002)
    for i in range (-1000,1000):
        x=np.array(compute_saliency_map(tf.cast(input[window_start+i:window_start+2001+i],tf.float32),model))
        y=np.concatenate((np.zeros(1000+i),x,np.zeros(1001-i)))
        Y=np.vstack([Y,y])
    return(Y)